# Вопрос по выбору

In [89]:
import matplotlib.pyplot as plt
import ipywidgets
import numpy as np
import pandas as pd
import scipy as sp
from scipy.io import wavfile
from scipy import signal
from scipy.signal import periodogram as periodogram_f
from scipy.fft import fftfreq, fftshift
from scipy.fft import fft, ifft, fft2, ifft2

### Рассчет RC фильтра нижних частот

Для регулировки частоты среза будет использоваться потенциометр на $10 \ кОм$. Максимальное сопротивление данного потенциометра - $9.2 \ кОм$(измерено при помощи мультиметра). Нужно подобрать емкость конденсатора так, чтобы частота среза удобно регулировалась в пределах слухового частотного диапазона.

АЧХ фильтра:
$$
U_{out} = U_{in} \frac{X_{c}}{\sqrt{R^2 + X_{C}^2}} \ , X_{C} = \frac{1}{2 \pi \nu C}
$$
Частота среза:
$$
f_{cutoff} = \frac{1}{2 \pi RC}
$$
Построим АЧХ и подберём параметры  цепи:

In [90]:
#ёмкость измеряем в нанофарадах, сопротивление в КОМ

def plotAFR(Resis = 0.1,C = 100):
    fig,ax = plt.subplots(figsize = (16,9))
    plt.grid()
    plt.xlabel("Частота $Гц$")
    plt.ylabel("Амплитуда")
    plt.xlim(20,22000)
    plt.xticks(np.arange(20,22000,1000))
    plt.ylim(0,1)
    R = Resis*1000
    cutoff = 1/(2*np.pi * R* C)
    freq = np.linspace(20,22000,22000) #аудиодиапазон
    X_c = 1/(2*np.pi*freq*(C*np.power(0.1,9)))
    amp = R/np.sqrt(np.power(R,2)+np.power(X_c,2))
    plt.plot(freq,amp,color = 'blue')
ipywidgets.interact(plotAFR,Resis = (0.1,10,0.2),C = (0.1,500,5))

interactive(children=(FloatSlider(value=0.1, description='Resis', max=10.0, min=0.1, step=0.2), FloatSlider(va…

<function __main__.plotAFR(Resis=0.1, C=100)>

<<Хорошей>> ёмкостью оказалась $50 \ n F$. Ближайшее значение ёмкости, которое у меня было - $0.1 \ \mu F$. Данное значение было выбрано как для LPF, так и для HPF.

In [58]:
C: float = 0.1 * np.power(0.1,6) #выбранное значение емкости

### Теоретическая модель

Коэффициент усиления:
$$
k = 1 + \frac{100}{4.7} = 22.3
$$
При меньших коэффициентах усиления схема переставала <<самовозбуждаться>>, и амплитуда выходного сигнала была сложно уловима диктофоном с динамическим диапазоном $96 \ dB$.

In [59]:
ampl_k = 22.3

In [87]:
def plotAFR(R_hpf = 1, R_lpf = 1):
    fig, ax = plt.subplots(figsize = (16,9))
    plt.grid()
    plt.xlabel("Частота $Гц$")
    plt.ylabel("Амплитуда")
    plt.xlim(20,22000)
    plt.xticks(np.arange(20,22000,1000))
    plt.ylim(0,int(ampl_k))
    ####
    R1 = R_hpf*1000
    R2 = R_lpf*1000
    freq = np.linspace(20,22000,20000)
    X_c = 1/(2*np.pi*freq*C)
    A_hpf = R1/np.sqrt(np.power(X_c,2)+np.power(R1,2))
    A_lpf = X_c/np.sqrt(np.power(X_c,2)+np.power(R2,2))
    A_result = ampl_k*A_hpf*A_lpf
    plt.plot(freq,A_hpf,label = "Фильтр высоких частот")
    plt.plot(freq,A_lpf,label = 'Фильтр низких частот')
    plt.plot(freq,A_result,label = 'Результат',color = "red")
    plt.legend()


In [88]:
ipywidgets.interact(plotAFR,R_hpf = (0.1,10,0.2),R_lpf = (0.1,10,0.2))

interactive(children=(FloatSlider(value=1.0, description='R_hpf', max=10.0, min=0.1, step=0.2), FloatSlider(va…

<function __main__.plotAFR(R_hpf=1, R_lpf=1)>

### Эксперимент

Попытка 1 - пробная

In [ ]:
rate, data = sp.io.wavfile.read('../audio/DR0000_0229.wav')
data=data[:,0]
time = np.arange(0, len(data), 1)/rate